<a href="https://colab.research.google.com/github/DCDPUAEM/DCDP/blob/main/03-Deep-Learning/notebooks/03-MLP-Regresion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Redes Neuronales MLP para regresión

✅ Conectar la notebook en modo GPU (**en este caso no es muy necesario**)

Entorno de ejecución → Cambiar tipo de entorno de ejecución

Algunas consideraciones:

* No dejar la notebook conectada sin actividad ya que Colab penaliza esto al asignar un entorno con GPU.
* No pedir el entorno con GPU si no se va a usar.

En esta notebook describiremos cómo resolver un problema de regresión usando una red neuronal MLP.

Usaremos el conjunto de datos [Auto MPG](https://archive.ics.uci.edu/ml/datasets/auto+mpg) y construiremos un modelo para predecir la eficiencia en el uso de combustible (en MPG, millas por galón) de vehiculos hechos entre 1970 y 1980. La descripción de cada vehículo incluye atributos como: número de cilíndros, potencia, país de origen y peso.

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## El conjunto de datos

El dataset original se puede encontrar en [UCI Machine Learning Repository](archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data).


In [ ]:
url = "https://raw.githubusercontent.com/DCDPUAEM/DCDP/main/03-Deep-Learning/data/auto-mpg.data"
!wget --no-cache --backups=1 {url}

Leemo el conjunto de datos en un dataframe.

In [ ]:
column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight',
                'Acceleration', 'Model Year', 'Origin']
df = pd.read_csv('auto-mpg.data', names=column_names,
                      na_values = "?", comment='\t',
                      sep=" ", skipinitialspace=True)
df

### Limpieza de los datos

El dataset contiene algunos valores desconocidos.

In [ ]:
df.isna().sum()

Eliminemos las filas con valores faltantes, ya que son pocas. También podríamos imputar valores.

In [9]:
clean_df = df.dropna()

La columna `"Origin"` es categorica, la codificamos con "one-hot" encoding

In [ ]:
oh_df = pd.get_dummies(data=clean_df,columns=['Origin'])
oh_df

Reemplazamos los nombres del origen

In [ ]:
oh_df.rename(columns={'Origin_1':'USA',
                      'Origin_2':'Europe',
                      'Origin_3':'Japan'},
             inplace=True)
oh_df

Separamos las features y la variable dependiente.

In [ ]:
X = oh_df.iloc[:,1:].values
y = oh_df['MPG'].values

print(X.shape, y.shape)

### División en conjuntos de entrenamiento y prueba

Ahora dividimos el set de datos en un set de entrenamiento y otro de prueba.

Usaremos el conjunto de prueba en la evaluacion final de nuestro modelo.

In [13]:
from tensorflow.python import train
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.85,random_state=189)

print(f"Train size: {X_train.shape[0]}")
print(f"Test size: {X_test.shape[0]}")  

### Normalizamos

Inspeccionemos los rangos de las variables continuas

In [ ]:
oh_df.iloc[:,1:7].plot.hist(subplots=True, legend=True)

Es una buena práctica normalizar funciones que utilizan diferentes escalas y rangos. Aunque el modelo *podría* converger sin normalización de features, esto suele dificultar el entrenamiento.

**Observaciones**: 
1. Aunque sólo entrenamos el escalador con el conjunto de datos de entrenamiento, este escalador también se utilizará para normalizar el conjunto de datos de prueba. Necesitamos hacer eso para proyectar el conjunto de datos de prueba en la misma distribución en la que el modelo ha sido entrenado.
2. El reescalamiento debe aplicarse a cualquier otro dato que entre aal modelo, junto con la codificación de un punto que hicimos anteriormente. Eso incluye el conjunto de pruebas, así como los datos en vivo cuando el modelo se usa en producción.

In [15]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scl = scaler.fit_transform(X_train)
X_test_scl = scaler.transform(X_test) 

## El modelo

### Construye el modelo

Construyamos nuestro modelo. Aquí, utilizaremos un modelo `sequential` con dos capas ocultas y una capa de salida que devuelve un único valor continuo. 

Observa la elección de optimizador, métricas de rendimiento, función de costo y funciones de activación.

Podemos definir directamente el modelo, como en la notebook pasada:

In [16]:
model = keras.Sequential([
layers.Dense(64, activation='relu', input_shape=[X_train_scl.shape[1]]),
layers.Dense(64, activation='relu'),
layers.Dense(1)
])

optimizer = tf.keras.optimizers.RMSprop(0.001)

model.compile(loss='mse',
            optimizer=optimizer,
            metrics=['mae', 'mse'])

Podemos también definirlo por medio de una función para generar nuevos modelos similares posteriormente:

In [17]:
def build_model():
    model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[X_train_scl.shape[1]]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
    ])
    optimizer = tf.keras.optimizers.RMSprop(0.001)
    model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
    return model

### Inspeccione el modelo

Use el método `.summary` para imprimir una descripción simple del modelo

In [ ]:
model.summary()

### ⏸A little detour...

Observemos que ya podríamos realizar predicciones con el modelo sin entrenar. Es decir, los pesos están inicializados

Tomamos un *batch* de 10 ejemplos de los datos de entrenamiento y realizamos las predicciones.

In [ ]:
example_batch = X_train_scl[:10]
example_result = model.predict(example_batch)
example_result

Podríamos medir su error MSE, o cualquier otra métrica de rendimiento.

In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_train[:10],example_result)

### Entrenamos el modelo

Entrenamos el modelo durante 1000 épocas, registramos la precisión de entrenamiento y validación en el objeto `history`.

In [ ]:
EPOCHS = 1000

history = model.fit(
  X_train_scl, y_train,
  epochs=EPOCHS, validation_split = 0.2, verbose=1)

Podríamos visualizar la historia del entrenamiento durante cada época en un dataframe usando las estadísticas almacenadas en el diccionario `history.history`. 

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

Definimos la siguiente función para graficar las métricas de rendimiento durante el entrenamiento.

In [23]:
def plot_history(history):
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Abs Error [MPG]')
    plt.plot(history.epoch, history.history['mae'],
            label='Train Error')
    plt.plot(history.epoch, history.history['val_mae'],
            label = 'Val Error')
    plt.legend()

    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Square Error [$MPG^2$]')
    plt.plot(history.epoch, history.history['mse'],
            label='Train Error')
    plt.plot(history.epoch, history.history['val_mse'],
            label = 'Val Error')
    plt.legend()
    plt.show()

In [ ]:
plot_history(history)

Este gráfico muestra poca mejora, o incluso degradación en el error de validación después de aproximadamente 100 épocas. **Esta es una señal de overfitting**.

Repitamos el entrenamiento con menos épocas. 

Observa el parámetro `verbose`

In [ ]:
model_me = build_model()

EPOCHS = 100

history = model_me.fit(X_train_scl, y_train, epochs=EPOCHS,
                    validation_split = 0.2, verbose=0)

plot_history(history)

### Métricas de rendimiento

Veamos qué tan bien generaliza el modelo al usar el **conjunto de prueba**, el cual no fue usado para entrenar el modelo. Esto nos dice qué tan bien podemos esperar que el modelo prediga cuándo lo usamos en el mundo real.

In [ ]:
loss, mae, mse = model_me.evaluate(X_test_scl, y_test, verbose=2)

print(f"MAE para las predicciones en el conjunto de prueba: {np.round(mae,4)} MPG")

### Predicciones

Finalmente, prediga los valores de MPG utilizando datos en el conjunto de pruebas:

In [ ]:
y_pred = model_me.predict(X_test_scl).flatten()

plt.scatter(y_test, y_pred)
plt.xlabel('True Values [MPG]')
plt.ylabel('Predictions [MPG]')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_ = plt.plot([-100, 100], [-100, 100])


Parece que nuestro modelo predice razonablemente bien. Echemos un vistazo a la distribución de errores.

In [ ]:
error = y_pred - y_test
plt.hist(error, bins = 25)
plt.xlabel("Error en la predicción [MPG]")
plt.ylabel("Conteos")
plt.show()

## ⭕ Práctica

1. Repite el experimento usando 100 épocas y sin normalizar los datos, ¿qué le sucede a las métricas de rendimiento?

2. Usando los datos normalizados, cambia los parámetros del módelo: 
    * Número de capas ocultas
    * Funciones de activación de las capas ocultas
    * Optimizador
    
    ¿Puedes subir las métricas de rendimiento?

3. ¿Mejora el rendimiento de tu modelo si imputas los 6 valores faltantes?

## Conclusiones

* El error cuadrático medio (MSE) es una función de pérdida común utilizada para problemas de regresión. Otra métrica de regresión común es el error absoluto medio (MAE). 
* Cuando las features de datos de entrada numéricos tienen valores con diferentes rangos, cada característica debe escalarse independientemente al mismo rango.
* Si no hay muchos datos de entrenamiento, es preferible usar una red pequeña con pocas capas ocultas para evitar el sobreajuste.
* El entrenamiento con pocas épocas es una técnica útil para evitar el sobreajuste. Otra técnica es el *early stopping** (coming soon...).